In [10]:
import os
import numpy as np

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger

#warnings.filterwarnings("ignore")
#tf version should be 2.9.0
print(tf.__version__)

2.9.0


In [11]:
rand_seed = 42

In [12]:
dir_path = r"C:\Users\acer\Desktop\Data_2D3layers\data_x"
results_dir = os.path.join(dir_path, r'logs\EfficientNet-B5')
models_dir = os.path.join(dir_path, r'models\EfficientNet-B5')

if not os.path.isdir(results_dir):
    os.makedirs(results_dir)
    
if not os.path.isdir(models_dir):
    os.makedirs(models_dir)

In [13]:
# define the location of your dataset

TRAIN_PATH = r"C:\Users\acer\Desktop\Data_2D3layers\data_x\train"
TRAIN_LABEL_PATH = r"C:\Users\acer\Desktop\Data_2D3layers\train_classification.csv"

VAL_PATH = r"C:\Users\acer\Desktop\Data_2D3layers\data_x\validation"
VAL_LABEL_PATH = r"C:\Users\acer\Desktop\Data_2D3layers\validation_classification.csv"

IMG_DIM = (224,224)
INPUT_SHAPE = (224,224,3)

BATCH_SIZE = 32
EPOCH = 20
x_axis_inc = 1 # for plotting the training acc and loss

In [14]:
import pandas as pd

train_original_df = pd.read_csv(TRAIN_LABEL_PATH)
train_original_df.head()

,ID,OA_normal_Remodel,c_erosion,c_subCyst,c_genSclerosis,c_osteophyte,c_flattening,c_corticalSclerosis
0,63-2829 L,-1,0,0,0,0,1,0
1,58-32020 R,1,1,0,0,1,1,1
2,54-25927 R,-1,0,0,0,0,1,0
3,66-218 R 2023 06 16,-1,0,0,0,0,1,0
4,60-700919 L,-1,0,0,0,0,1,1


In [15]:
def expand_dataframe(df):
    # Create a list to hold the new rows
    new_rows = []
    
    # Iterate through each row in the original dataframe
    for _, row in df.iterrows():
        # Duplicate the row 234 times (for indices 0-233)
        for i in range(224):
            new_row = row.copy()
            new_row['ID'] = f"{row['ID']}_x_{i:03d}"  # Wrap the original ID and add suffix
            new_rows.append(new_row)
    
    # Create a new dataframe from the list of new rows
    new_df = pd.DataFrame(new_rows)
    
    return new_df

train_original_df = train_original_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)

train_c_erosion_label = train_original_df[['ID','c_erosion']]
# Create the new DataFrame
train_df = expand_dataframe(train_c_erosion_label)

# Shuffle the training data
#train_df = train_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)

train_df.head()

,ID,c_erosion
0,66-4559 L_x_000,1
0,66-4559 L_x_001,1
0,66-4559 L_x_002,1
0,66-4559 L_x_003,1
0,66-4559 L_x_004,1


In [16]:
slice_number = '160'
train_df = train_df[train_df['ID'].str.endswith(slice_number)]
train_df.head()

,ID,c_erosion
0,66-4559 L_x_160,1
1,51-28114 L_x_160,1
2,57-24983 L_x_160,0
3,61-25724 L_x_160,1
4,57-34770 L_x_160,0


In [17]:
data_labels = train_df
target_labels = data_labels['c_erosion']
data_labels['image_path'] =  data_labels.apply(lambda row: (os.path.join(TRAIN_PATH, str(row['ID'])) + '.jpg'), axis=1)
data_labels.head()

,ID,c_erosion,image_path
0,66-4559 L_x_160,1,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...
1,51-28114 L_x_160,1,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...
2,57-24983 L_x_160,0,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...
3,61-25724 L_x_160,1,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...
4,57-34770 L_x_160,0,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...


In [18]:
print(data_labels.shape)

(254, 3)


In [19]:
import pandas as pd
import os

def check_valid_files(df, column_name='image_path'):
    # Create a new column for validity
    df['is_valid_file'] = df[column_name].apply(os.path.isfile)
    
    # Print summary
    total = len(df)
    valid = df['is_valid_file'].sum()
    invalid = total - valid
    
    print(f"Total files: {total}")
    print(f"Valid files: {valid}")
    print(f"Invalid files: {invalid}")
    
    # If there are invalid files, you can get them like this:
    if invalid > 0:
        print("\nInvalid files:")
        invalid_files = df[~df['is_valid_file']][column_name]
        for file in invalid_files:
            print(file)
    
    return df

check_valid_file_df = check_valid_files(data_labels)

# You can access the results in the DataFrame
check_valid_file_df.head(10)

Total files: 254
Valid files: 254
Invalid files: 0


,ID,c_erosion,image_path,is_valid_file
0,66-4559 L_x_160,1,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
1,51-28114 L_x_160,1,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
2,57-24983 L_x_160,0,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
3,61-25724 L_x_160,1,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
4,57-34770 L_x_160,0,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
5,61-20310 R_x_160,1,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
6,60-2688 L_x_160,1,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
7,62-1401 L_x_160,0,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
8,57-29097 L_x_160,1,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True
9,55-1648 R_x_160,0,C:\Users\acer\Desktop\Data_2D3layers\data_x\tr...,True


In [20]:
val_original_df = pd.read_csv(VAL_LABEL_PATH)
val_c_erosion_label = val_original_df[['ID','c_erosion']]
# Create the new DataFrame
val_df = expand_dataframe(val_c_erosion_label)

# Shuffle the validation data
val_df = val_df.sample(frac=1, random_state=rand_seed).reset_index(drop=True)

# Filter for only the desired slice (ened with 'slice_nunber')
val_df = val_df[val_df['ID'].str.endswith(slice_number)]

val_df.head()

,ID,c_erosion
60,65-2425 R 2023_x_160,0
105,62-700851 L_x_160,0
197,63-20205 R_x_160,1
444,65-25886 R_x_160,0
652,60-16447 L_x_160,1


In [21]:
val_labels = val_df[['ID','c_erosion']]
target_val_labels = val_labels['c_erosion']
val_labels['image_path'] =  val_labels.apply(lambda row: (os.path.join(VAL_PATH, str(row['ID'])) + '.jpg'), axis=1)
val_labels.head()

,ID,c_erosion,image_path
60,65-2425 R 2023_x_160,0,C:\Users\acer\Desktop\Data_2D3layers\data_x\va...
105,62-700851 L_x_160,0,C:\Users\acer\Desktop\Data_2D3layers\data_x\va...
197,63-20205 R_x_160,1,C:\Users\acer\Desktop\Data_2D3layers\data_x\va...
444,65-25886 R_x_160,0,C:\Users\acer\Desktop\Data_2D3layers\data_x\va...
652,60-16447 L_x_160,1,C:\Users\acer\Desktop\Data_2D3layers\data_x\va...


In [22]:
check_valid_file_df = check_valid_files(val_labels)
check_valid_file_df.head()

Total files: 54
Valid files: 54
Invalid files: 0


,ID,c_erosion,image_path,is_valid_file
60,65-2425 R 2023_x_160,0,C:\Users\acer\Desktop\Data_2D3layers\data_x\va...,True
105,62-700851 L_x_160,0,C:\Users\acer\Desktop\Data_2D3layers\data_x\va...,True
197,63-20205 R_x_160,1,C:\Users\acer\Desktop\Data_2D3layers\data_x\va...,True
444,65-25886 R_x_160,0,C:\Users\acer\Desktop\Data_2D3layers\data_x\va...,True
652,60-16447 L_x_160,1,C:\Users\acer\Desktop\Data_2D3layers\data_x\va...,True


In [23]:
train_data_size = 254
validation_data_size = 54

In [24]:
train_data = np.array([img_to_array(load_img(img, target_size=IMG_DIM))
                       for img in data_labels['image_path'][0:train_data_size].values.tolist()]).astype('float32')

In [25]:
val_data = np.array([img_to_array(load_img(img, target_size=IMG_DIM))
                       for img in val_labels['image_path'][0:validation_data_size].values.tolist()]).astype('float32')

In [26]:
print('Training Dataset Size:', train_data.shape)
print('Validation Dataset Size:', val_data.shape)

Training Dataset Size: (254, 224, 224, 3)
Validation Dataset Size: (54, 224, 224, 3)


## Prepare for Transfer Learning

In [27]:
prep_in = tf.keras.applications.efficientnet.preprocess_input

In [28]:
x_train_new = train_data.astype('int')
x_val_new = val_data.astype('int')

In [29]:
x_train_in = prep_in(x_train_new)
x_val_in = prep_in(x_val_new)

In [30]:
train_labels_enc = target_labels[0:train_data_size].to_numpy()
val_labels_enc = target_val_labels[0:validation_data_size].to_numpy()

In [31]:
# Get the VGG16 model so we can do transfer learning
base_model = tf.keras.applications.EfficientNetB5(input_shape=INPUT_SHAPE, include_top=False, weights='imagenet')

In [32]:
print('Number of layers in the base model: ', len(base_model.layers))

Number of layers in the base model:  577


In [33]:
pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at 0x000002824FD6C400>,input_1,True
1,<keras.layers.preprocessing.image_preprocessing.Rescaling object at 0x000002824FF869D0>,rescaling,True
2,<keras.layers.preprocessing.normalization.Normalization object at 0x00000282500B68E0>,normalization,True
3,<keras.layers.core.tf_op_layer.TFOpLambda object at 0x00000282500B6940>,tf.math.truediv,True
4,<keras.layers.reshaping.zero_padding2d.ZeroPadding2D object at 0x00000282500B6130>,stem_conv_pad,True
...,...,...,...
572,<keras.layers.regularization.dropout.Dropout object at 0x00000282537C11F0>,block7c_drop,True
573,<keras.layers.merging.add.Add object at 0x0000028251A94CA0>,block7c_add,True
574,<keras.layers.convolutional.conv2d.Conv2D object at 0x000002824FD7FC10>,top_conv,True
575,<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000282505ED670>,top_bn,True


## Slapping AvgPool > 1024_Dense > 512_Dense > 1_output on top of the EfficientNet

In [34]:
base_model.trainable = False

pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in base_model.layers[10:]]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.layers.core.activation.Activation object at 0x000002824FD270A0>,block1a_activation,False
1,<keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D object at 0x000002824FF3BF70>,block1a_se_squeeze,False
2,<keras.layers.reshaping.reshape.Reshape object at 0x000002824FD49160>,block1a_se_reshape,False
3,<keras.layers.convolutional.conv2d.Conv2D object at 0x0000028250226A90>,block1a_se_reduce,False
4,<keras.layers.convolutional.conv2d.Conv2D object at 0x000002824FA89CD0>,block1a_se_expand,False
...,...,...,...
562,<keras.layers.regularization.dropout.Dropout object at 0x00000282537C11F0>,block7c_drop,False
563,<keras.layers.merging.add.Add object at 0x0000028251A94CA0>,block7c_add,False
564,<keras.layers.convolutional.conv2d.Conv2D object at 0x000002824FD7FC10>,top_conv,False
565,<keras.layers.normalization.batch_normalization.BatchNormalization object at 0x00000282505ED670>,top_bn,False


In [35]:
fig_train = 'EfficientNetB5_A_FE_train_224'
log_file = os.path.join(results_dir, 'EfficientNetB5_A_FE_train_224.csv')
model_path = os.path.join(models_dir, 'EfficientNetB5_A_FE_224.h5')

In [36]:
# Create a new model on top

inputs = tf.keras.Input(shape=INPUT_SHAPE)
# We make sure that the base_model is running in inference mode here,
# by passing 'training=False'. This is important for fine-tuning
x = base_model(inputs, training=False)

x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
x = Dense(512, activation='relu')(x)

# A Dense classifier with a single unit (binary classification)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs, outputs)

In [37]:
pd.set_option('max_colwidth', None)
layers = [(layer, layer.name, layer.trainable) for layer in model.layers]
pd.DataFrame(layers, columns=['Layer Type', 'Layer Name', 'Layer Trainable'])

,Layer Type,Layer Name,Layer Trainable
0,<keras.engine.input_layer.InputLayer object at 0x000002825394FFD0>,input_2,True
1,<keras.engine.functional.Functional object at 0x00000282537D1A30>,efficientnetb5,False
2,<keras.layers.pooling.global_average_pooling2d.GlobalAveragePooling2D object at 0x000002825124AB50>,global_average_pooling2d,True
3,<keras.layers.core.dense.Dense object at 0x00000282524511C0>,dense,True
4,<keras.layers.core.dense.Dense object at 0x0000028251A9E8E0>,dense_1,True
5,<keras.layers.core.dense.Dense object at 0x0000028252AAB640>,dense_2,True


In [38]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 efficientnetb5 (Functional)  (None, 7, 7, 2048)       28513527  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1024)              2098176   
                                                                 
 dense_1 (Dense)             (None, 512)               524800    
                                                                 
 dense_2 (Dense)             (None, 1)                 513       
                                                             

In [39]:
callbacks = [
    CSVLogger(log_file),
    ModelCheckpoint(
        filepath=os.path.join(models_dir,"Eff-epoch-{epoch:02d}.h5"),
        save_weights_only=False,
        save_best_only=False,
        save_freq='epoch',
        verbose=1
    )
]

In [40]:
model.compile(loss=keras.losses.BinaryCrossentropy(from_logits=False), # change from_logits=True
              optimizer= keras.optimizers.RMSprop(learning_rate=1e-3),
              metrics = [keras.metrics.BinaryAccuracy()])

In [41]:
history = model.fit(x=x_train_in, 
                    y=train_labels_enc,
                    validation_data=(x_val_in, val_labels_enc),
                    epochs=EPOCH, 
                    verbose=1, 
                    callbacks=callbacks)

Epoch 1/20
8/8 [==============================] - ETA: 0s - loss: 1.4950 - binary_accuracy: 0.5709
Epoch 1: saving model to C:\Users\acer\Desktop\Data_2D3layers\data_x\models\EfficientNet-B5\Eff-epoch-01.h5
8/8 [==============================] - 37s 3s/step - loss: 1.4950 - binary_accuracy: 0.5709 - val_loss: 0.6371 - val_binary_accuracy: 0.7407
Epoch 2/20
8/8 [==============================] - ETA: 0s - loss: 0.6596 - binary_accuracy: 0.6575
Epoch 2: saving model to C:\Users\acer\Desktop\Data_2D3layers\data_x\models\EfficientNet-B5\Eff-epoch-02.h5
8/8 [==============================] - 24s 3s/step - loss: 0.6596 - binary_accuracy: 0.6575 - val_loss: 0.6294 - val_binary_accuracy: 0.6481
Epoch 3/20
8/8 [==============================] - ETA: 0s - loss: 0.5397 - binary_accuracy: 0.7323
Epoch 3: saving model to C:\Users\acer\Desktop\Data_2D3layers\data_x\models\EfficientNet-B5\Eff-epoch-03.h5
8/8 [==============================] - 24s 3s/step - loss: 0.5397 - binary_accuracy: 0.7323 - val